In [1]:
from neuronunit.tests.druckmann2013 import AP1AmplitudeTest, AP1DelayMeanStrongStimTest
from neuronunit.models.neuron_cell import NeuronCellModel
import os
import requests
import quantities as pq

/home/justas/anaconda2/envs/p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Could not load NEURONBackend


In [2]:
def download_extract_compile_model(nmldb_modelid):
    # Get info about the model from NMLDB API
    model_info_url = "https://neuroml-db.org/api/model?id="+nmldb_modelid
    model_info = requests.get(model_info_url).json()

    # Download model from NMLDB
    os.system('wget -O model.zip "https://neuroml-db.org/GetModelZip?modelID='+nmldb_modelid+'&version=NeuroML"')

    # Unzip the model to sub dir
    os.system("unzip -o model.zip -d " + nmldb_modelid)

    # Change the CWD to the model dir
    os.chdir(nmldb_modelid)

    # Convert NML to NEURON using jNeuroML
    cell_file_name = model_info["model"]["File_Name"]

    os.system("jnml "+cell_file_name+" -neuron -nogui")

    # Compile any .mod files
    os.system("nrnivmodl")
    
    return model_info


def get_hoc_files():
    return [f for f in os.listdir('.') if f.endswith(".hoc")]

def get_mod_files():
    return [f for f in os.listdir('.') if f.endswith(".mod")]
    
def is_abstract_cell():
    return len(get_hoc_files()) == 0 and len(get_mod_files()) == 1

def get_abstract_cell(h):
    cell_mod_file = get_mod_files()[0]
    cell_mod_name = cell_mod_file.replace(".mod", "")

    soma = h.Section()
    soma.L = 10
    soma.diam = 10
    soma.cm = 318.31927  # Magic number, see: https://github.com/NeuroML/org.neuroml.export/issues/60

    mod = getattr(h, cell_mod_name)(0.5, sec=soma)

    return soma, mod

def get_cell_with_morphology(h):
    cell_hoc_file = get_hoc_files()[0]
    cell_template = cell_hoc_file.replace(".hoc", "")
    h.load_file(cell_hoc_file)
    cell = getattr(h, cell_template)()
    return cell

def get_soma(h):
    # Get the root sections and try to find the soma
    roots = h.SectionList()
    roots.allroots()
    roots = [s for s in roots]
    somas = [sec for sec in roots if "soma" in sec.name().lower()]
    if len(somas) == 1:
        return somas[0]
    elif len(somas) == 0 and len(roots) == 1:
        return roots[0]
    else:
        raise Exception("Problem finding the soma section")

In [3]:
# Izhikevich Class I model. For more: https://neuroml-db.org/search_model?q=Izhikevich
model_info = download_extract_compile_model(nmldb_modelid = 'NMLCL001413')

In [4]:
# Load model in NEURON
from neuron import h,gui

if is_abstract_cell():
    cell, mod = get_abstract_cell(h)
else:
    cell = get_cell_with_morphology(h)

soma = get_soma(h)

h.cvode_active(1)

1.0

In [5]:
# Create a NeuronUnit model from the NEURON model
name = model_info["model"]["Name"]
temp = model_info["publication"]["record"]["Temperature"]

cell_model = NeuronCellModel(soma(0.5), name=name)
cell_model.set_stop_time(3000*pq.ms)
cell_model.set_temperature(temp)

cell_model.default_sampling_period = 0.25 # Izh APs are very narrow

h.newPlotV() # optional

0.0

In [6]:
#Get the Druckmann2013 "standard" and "strong" currents
rheobase = model_info["model"]["Rheobase_High"] * pq.nA

standard = rheobase * 1.5
strong = rheobase * 3.0

temp = model_info["publication"]["record"]["Temperature"]

In [7]:
# Druckmann standard current test
test = AP1AmplitudeTest(current_amplitude=standard, observation={"mean":30*pq.mV,"std":5*pq.mV, "n":1})
test.generate_prediction(cell_model)

{'mean': array([40.95557726]) * mV, 'n': 1, 'std': 0}

In [8]:
# Another test using the strong current
test = AP1DelayMeanStrongStimTest(current_amplitude=strong, observation={"mean":100*pq.ms,"std":5*pq.ms, "n":1})
test.params["repetitions"] = 1

#This one takes a long time
test.generate_prediction(cell_model)

{'mean': array(4.5) * ms, 'n': 7, 'std': array(0.) * ms}